In [45]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, roc_auc_score
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

In [46]:
bb = pd.read_csv('blueberry_train.csv',index_col=0)

,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,yield
id,,,,,,,,,,,,,,,,,
0,25.0,0.50,0.25,0.75,0.50,69.7,42.1,58.2,50.2,24.3,41.2,24.0,0.39,0.425011,0.417545,32.460887,4476.81146
1,25.0,0.50,0.25,0.50,0.50,69.7,42.1,58.2,50.2,24.3,41.2,24.0,0.39,0.444908,0.422051,33.858317,5548.12201
2,12.5,0.25,0.25,0.63,0.63,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.552927,0.470853,38.341781,6869.77760
3,12.5,0.25,0.25,0.63,0.50,77.4,46.8,64.7,55.8,27.0,45.8,24.0,0.39,0.565976,0.478137,39.467561,6880.77590
4,25.0,0.50,0.25,0.63,0.63,77.4,46.8,64.7,55.8,27.0,45.8,24.0,0.39,0.579677,0.494165,40.484512,7479.93417


In [47]:
X = bb.drop(columns='yield')
y = bb['yield'] 

In [60]:
lr = LinearRegression()
dtr = DecisionTreeRegressor(random_state=24)
ss,mm = StandardScaler(), MinMaxScaler()

baggr = BaggingRegressor(estimator=dtr, random_state=24)
# pipe = Pipeline([('SC',None),('BAGGR',baggr)])
baggr.get_params()
kfold = KFold(n_splits=5, shuffle=True, random_state=24)
params = {'n_estimators' : [10,20,50],
         # 'SC':[None,ss,mm],
         'estimator__min_samples_leaf' : [1,10,20], 
         'estimator__min_samples_split' : [5,10,20],
         'estimator__max_depth' : [None, 2, 3]}
gcv = GridSearchCV(baggr, param_grid=params, cv= kfold, scoring='r2',verbose=3)
gcv.fit(X,y)


Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV 1/5] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=5, n_estimators=10;, score=0.799 total time=   0.6s
[CV 2/5] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=5, n_estimators=10;, score=0.800 total time=   0.7s
[CV 3/5] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=5, n_estimators=10;, score=0.795 total time=   0.6s
[CV 4/5] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=5, n_estimators=10;, score=0.795 total time=   0.6s
[CV 5/5] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=5, n_estimators=10;, score=0.801 total time=   0.6s
[CV 1/5] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=5, n_estimators=20;, score=0.807 total time=   1.3s
[CV 2/5] END estimator__ma

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=BaggingRegressor(estimator=DecisionTreeRegressor(random_state=24),
                                        random_state=24),
             param_grid={'estimator__max_depth': [None, 2, 3],
                         'estimator__min_samples_leaf': [1, 10, 20],
                         'estimator__min_samples_split': [5, 10, 20],
                         'n_estimators': [10, 20, 50]},
             scoring='r2', verbose=3)

In [61]:
print("Best Score: ",gcv.best_score_)
print("Best Params: ",gcv.best_params_)

Best Score:  0.8202763790638787
Best Params:  {'estimator__max_depth': None, 'estimator__min_samples_leaf': 20, 'estimator__min_samples_split': 5, 'n_estimators': 50}


In [63]:
tst = pd.read_csv('blueberry_test.csv',index_col=0)
blueberry_test = gcv.best_estimator_
y_pred = blueberry_test.predict(tst)

submit = pd.read_csv('blueberry_sample_submission.csv')
submit.price = y_pred
submit.to_csv('Submission_BlueBerry_yield_WithDTR_Predictiion.csv', index=False)